In [ ]:
library(data.table)
library(rethinking)

In [ ]:
options(mc.cores = parallel::detectCores())

In [ ]:
setwd('~/go_nuts/')

In [ ]:
# Load trials data
trials1 = fread('~/go_nuts/trials022319.csv', 
            sep = '\t',
            header = FALSE,
            col.names = c('start', 'surface', 'outcome'))
trials2 = fread('~/go_nuts/trials022409.csv',
                sep = '\t',
                header = FALSE,
                col.names = c('start', 'surface', 'outcome'))
trials3 = fread('~/go_nuts/trials022519.csv',
                sep = '\t',
                header = FALSE,
                col.names = c('start', 'surface', 'outcome'))
trials4 = fread('~/go_nuts/trials02271408.csv',
                sep = '\t',
                header = FALSE,
                col.names = c('start', 'surface', 'outcome'))
cat('trials1:\n')
trials1[, .N, by = .(start, surface, outcome)]
cat('trials2:\n')
trials2[, .N, by = .(start, surface, outcome)]
cat('trials3:\n')
trials3[, .N, by = .(start, surface, outcome)]
cat('trials4:\n')
trials4[, .N, by = .(start, surface, outcome)]

trials = rbind(
  trials1, 
  trials2, 
  trials3, 
  trials4
)
cat('all trials:\n')
trials[, .N, by = .(start, surface)]

In [ ]:
# Add derived outcome, wherever it equals the start
trials[, outcome_eq_start := as.integer(outcome == start)]
cat('Table on outcome_eq_start:\n')
trials[, .N, by = outcome_eq_start]

In [ ]:
# Add 0/1 columns for boolean variables:
trials[, start_h := as.integer(start == 'H')]
trials[, surface_c := as.integer((surface == 'C'))]
trials[, outcome_h := as.integer(outcome == 'H')]
print(head(trials))

In [ ]:
d <- trials

In [ ]:
# Must convert to integer so Stan can treat properly:
# stone,tails ; stone,heads ; cloth,tails ; cloth,heads
d$treatment <- as.integer(1 + d$start_h + 2*d$surface_c)
head(d)

In [ ]:
# Try a very flat prior on the intercept, a:
m11.1coin_flatprior <- quap(
    alist(
        outcome_h ~ dbinom(1, p),
        logit(p) <- a,
        a ~ dnorm(0, 10)
    ),
    data = d
)
set.seed(1999)
prior <- extract.prior(m11.1coin_flatprior, n = 1e4)

p <- inv_logit(prior$a)

dens(p, adj=0.1, main = 'With flat prior, `prior$a` suggests all heads or all tails.')

# With a regularizing prior, we encode the information that even an adulterated coin has p
# centered around .5 -- it has the side-effect of making the sampler much more efficient, later.
m11.1coin <- quap(
    alist(
        outcome_h ~ dbinom(1, p),
        logit(p) <- a,
        a ~ dnorm(0, 1.5)
    ),
    data = d
)
set.seed(1999)
prior <- extract.prior(m11.1coin, n = 1e4)

p <- inv_logit(prior$a)

dens(p, adj=0.1, main = 'With gently regularizing prior, the prior-only predictions seem better.', 
     col = 'dodgerBlue')

In [ ]:
m11.2coin <- quap(
    alist(
        outcome_h ~ dbinom(1, p),
        logit(p) <- a + b[treatment],
        a ~ dnorm(0, 1.5),
        b[treatment] ~ dnorm(0, 10)
    ),
    data = d
)
set.seed(1999)
prior <- extract.prior(m11.2coin, n = 1e4)
num_treatments = length(unique(d$treatment))
cat(paste('Number of treatments:', num_treatments))
p <- sapply(1:(num_treatments - 1), function(k) inv_logit(prior$a + prior$b[,k]))

dens(abs(p[,1] - p[,2]), adj = 0.1)

In [ ]:
m11.3coin_better <- quap(
    alist(
        outcome_h ~ dbinom(1, p),
        logit(p) <- a + b[treatment],
        a ~ dnorm(0, 1.5),
        b[treatment] ~ dnorm(0, .5)
    ),
    data = d
)
set.seed(1999)
prior <- extract.prior(m11.3coin_better, n = 1e4)
num_treatments = length(unique(d$treatment))
cat(paste('Number of treatments:', num_treatments))
p <- sapply(1:(num_treatments - 1), function(k) inv_logit(prior$a + prior$b[,k]))

dens(abs(p[,1] - p[,2]), adj = 0.1, 'Treatment effects now regularized in the prior', 
     col = 'dodgerBlue2')

In [ ]:
dat_list_simple <- list(
    outcome_h = d$outcome_h,
    surface_c1 = d$surface_c
)

In [ ]:
m11.4pop_only <- ulam(
    alist(
        outcome_h ~ dbinom(1, p),
        logit(p) <- a + b * surface_c1,
        a ~ dnorm(0, 1.5),
        b ~ dnorm(0, .5)
    ),
    data = dat_list_simple, 
    chains = 4, iter = 5000, warmup = 1000, log_lik = TRUE
)


In [ ]:
precis(m11.4pop_only, depth = 2)
plot(precis(m11.4pop_only, depth = 2))

In [ ]:
show(m11.4pop_only)

In [ ]:
cat('Number of posterior samples:',length(post$a))

In [ ]:
post <- extract.samples(m11.4pop_only)
plen <- length(post$a)
zeros <- floor(length(post$a)/2)
surface_synth <- c(rep(0, zeros), rep(1, plen-zeros))

In [ ]:
p_heads <- inv_logit(post$a + post$b * surface_synth)

In [ ]:
probs  <- list(
    surface_stone = inv_logit(post$a[surface_synth == 0] + 
                              post$b[surface_synth == 0] * surface_synth[surface_synth == 0]),
    surface_cloth = inv_logit(post$a[surface_synth == 1] + 
                              post$b[surface_synth == 1] * surface_synth[surface_synth == 1])  
)

In [ ]:
plot(precis(as.data.table(probs)), labels = names(probs))
abline(v = .5, col = 'gray60')

In [ ]:
# PREPARE FOR MORE COMPLICATED MODEL
# prior trimmed data list
# Need surface column to also work as a subscript in R, 
# which means needs values from 1:2 not 0:1
dat_list <- list(
    outcome_h = d$outcome_h,
    surface_c1 = as.integer(d$surface_c + 1),
    treatment = d$treatment
)

In [ ]:
# Particles in many-dimensional space:
# Sampling... for this case, this takes time to compile the model,
# offload it to Stan for external processing, and reload the results.
m11.4coin <- ulam(
    alist(
        outcome_h ~ dbinom(1, p),
        logit(p) <- a[surface_c1] + b[treatment],
        a[surface_c1] ~ dnorm(0, 1.5),
        b[treatment] ~ dnorm(0, 0.5)
    ),
    data = dat_list, 
    chains = 4, iter = 5000, warmup = 1000, log_lik = TRUE
)
precis(m11.4coin, depth = 2)

In [ ]:
show(m11.4coin)

In [ ]:
post <- extract.samples(m11.4coin)
p_heads <- inv_logit(post$a)
colnames(p_heads) <- c('surface_stone', 'surface_cloth')
plot(precis(as.data.frame(p_heads)), xlim = c(0,1))
abline(v = .5, col = 'gray60')

In [ ]:
# 11.12 adapted
labs <- c('stone/start_tails', 'stone/start_heads', 'cloth/start_tails', 'cloth/start_heads')
plot(precis(m11.4coin, depth = 2, pars = 'b'), labels = labs, main = 'Treatment effects')
diffs <- list(
    'cl/sta_t-sto/sta_t' = post$b[,1] - post$b[,3],
    'cl/sta_h-sto/sta_h' = post$b[,2] - post$b[,4]
)
plot(precis(diffs), labels = c('cloth/start_t - stone/start_t', 'cloth/start_h - stone/start_h'),
     main = 'Contrasts between the cloth & stone treatments')

In [ ]:
# Let's compare the simpler, varying-intercepts 
compare(m11.4pop_only, m11.4coin)

In [ ]:
# Let's check the trace plots to see how our Markov chains did. No sign of 
traceplot(m11.4coin)

In [ ]:
# If there were problems sampling, we can plot Stan's fitted-model output.
# If there were serious sampling problems, we'd see some very lopsided plots here,
# instead of round or slanted probability clouds.
pairs(m11.4coin@stanfit)

In [ ]:
#11.14 adapted
pl <- by(dat_list$outcome_h, list(dat_list$surface_c1, dat_list$treatment), mean)


In [ ]:
# 11.15 adapted
plot(NULL, xlim = c(1, 8), ylim = c(0, 1), xlab = '',
    ylab = 'proportion heads outcome', xaxt='n', yaxt='n')
axis(2, at=c(0, 0.5, 1), labels = c(0, 0.5, 1))
abline(h=0.5, lty = 2)
for (j in 1:2) abline(v=(j-1)*4+4.5, lwd = 0.5)
for (j in 1:2) text((j-1)*4 + 2.5, 1.1, c('surface_stone', 'surface_cloth')[j], xpd = TRUE)
for (j in 1:2) {
    lines((j-1)*4 + c(1,3), pl[j, c(1,3)], lwd = 2, col = rangi2)
    lines((j-1)*4 + c(2,4), pl[j, c(2,4)], lwd = 2, col = rangi2)
}
points(1:8, t(pl), pch=16, col='white', cex = 1.7)
points(1:8, t(pl), pch = c(1, 1, 16, 16), col = rangi2, lwd=2)
yoff <- 0.01
text(1, pl[1,1]-yoff, 'S/T', pos=1, cex = 0.8)
text(2, pl[1,2]+yoff, 'S/H', pos=3, cex = 0.8)
text(3, pl[1,3]-yoff, 'C/T', pos=1, cex = 0.8)
text(4, pl[1,4]+yoff, 'C/H', pos=3, cex = 0.8)
mtext('observed proportions\n')

In [ ]:
d.pred <- list(
    treatment = c(1,2,3,4,1,2,3,4),
    surface_c1 = c(1,1,1,1,2,2,2,2)
)

link.m11.4coin <- link(m11.4coin, data = d.pred)

apply(link.m11.4coin, 2, mean)